In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
# node = sy.orchestra.launch(name="test-domain-helm", port=8800, dev_mode=True, reset=True, tail=True)

with server

In [3]:
# client = sy.login(port=8800, email="info@openmined.org", password="changethis")

Without server

In [4]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True, reset=True, n_consumers=3)
client = node.login(email="info@openmined.org", password="changethis")

CREATING A PRODUCER ON tcp://127.0.0.1:56244
CREATING A CONSUMER ON tcp://127.0.0.1:56243
Starting receival (5a6400527a01464cb10f8479085289b9)
spawning thread
CREATING A CONSUMER ON tcp://127.0.0.1:56243
Starting receival (93bb6789658c4e8eadeae08a16b7c20f)
spawning thread
CREATING A CONSUMER ON tcp://127.0.0.1:56243
Starting receival (c2c64d3268a6453eb1aae300318a9cc8)
spawning thread
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [5]:
# from gevent import monkey

In [6]:
# monkey.patch_all(thread=True)

In [7]:
# type(node.python_node.queue_manager.consumers["api_call"])

In [8]:
# node.python_node.queue_manager.consumers["api_call"][0].address

setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

# Setup syft functions

## Dataset

In [9]:
x = ActionObject.from_obj([1,2])
x_ptr = x.send(client)

## Batch function

In [10]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [11]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [12]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
    results = [x.wait().get() for x in jobs]
    return sum(results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [13]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request 1148660f85a34fb5b3c0ea5e9165b664 changes applied

In [14]:
job = client.code.process_all(x=x_ptr, blocking=False)
sleep(10)

setting to queue stash
set
Trying to send 
Received stuff (5a6400527a01464cb10f8479085289b9)
sent back confirmation (5a6400527a01464cb10f8479085289b9)
Message Received Successfully !
sent
CREATING A PRODUCER ON tcp://127.0.0.1:56244
appending to 43e35ecf9eb44e5baa02ae629974c447, 140683235508800 Launching jobs
FUNCTION LOG: Launching jobs
ID 43e35ecf9eb44e5baa02ae629974c447
LAUNCHING JOB process_batch
setting to queue stash
set
Trying to send 
Received stuff (93bb6789658c4e8eadeae08a16b7c20f)
sent back confirmation (93bb6789658c4e8eadeae08a16b7c20f)
Message Received Successfully !
sent
ID 43e35ecf9eb44e5baa02ae629974c447
ID 43e35ecf9eb44e5baa02ae629974c447
ID 43e35ecf9eb44e5baa02ae629974c447
LAUNCHING JOB process_batch
setting to queue stash
set
Trying to send 
Received stuff (c2c64d3268a6453eb1aae300318a9cc8)
sent back confirmation (c2c64d3268a6453eb1aae300318a9cc8)
Message Received Successfully !
sent
ID 43e35ecf9eb44e5baa02ae629974c447
ID 43e35ecf9eb44e5baa02ae629974c447
appending to

In [15]:
job.logs()

Launching jobs
starting aggregation
Done



In [18]:
job.subjobs[1].logs()

starting batch 2
done
done



In [27]:
job.wait().get()

5

# Next steps



- [x] seaweedfs
- - [ ] merge into normal code
- [ ] Multiple consumers (queues)
- [x] test flow with async main function
- [x]  up datastructures for jobs and logs
- - [x] set up reprs
- [ ] integrate helm code
- [ ] test on large machine

# OLD

In [ ]:


def user_func_process_all_11w12342534(domain, x):
    def process_all(domain, x):
        print(locals().keys())
        print(globals().keys())
        print('process_batch' in globals())
        results = []
        for elem in x:
            job = domain.launch_job(process_batch, x_, y_)
        return 1
    result = process_all(domain=domain, x=x)
    return result


In [ ]:
# exec("""
# print(locals())
# print('b')"""
#     )

In [ ]:
# res.get()

In [ ]:
s = """
def y():
    print('inner local scope', 'z' in locals().keys())
    print('inner global scope', 'z' in globals().keys())
    print(z)
"""

In [ ]:
_locals = {"z": 1}